In [ ]:
import torch
from torchvision import datasets, transforms

### Setting

In [ ]:
# Optimization Setting
AUTOCAST_FLAG = False
COMPILE_FLAG = False

# Data && Model Setting
## DataShape
size = 4 #4, 8, 16, '32'
## RandAugment
num_ops = 2
magnitude = 3 #3, 6, '9', 12
## Model
num_classes = 100
p = 0.1 #0.1, '0.2', 0.3, 0.4
## DataLoader
batch_size = 512
eval_batch_size = 1000
num_workers = 2
PIN_MEMORY_FLAG = True
PERSISTENT_WORKERS_FLAG = True

# Training Setting
epochs = 160
## loss function
lr = 1e-1
momentum = 0.9
weight_decay = 1e-4
## exponential scheduler
gamma = 0.98
## stage milestone
milestones = [40, 80, 120]

In [ ]:
root = '~/.pytorch/datasets/'
device = torch.device(
    f'cuda:{torch.cuda.device_count() - 2}' if torch.cuda.is_available() else 'cpu'
)
if device.type == 'cuda':
    torch.cuda.set_device(device)
    torch.backends.cudnn.benchmark = True
print(f'Device: {device}, Type: {device.type}')

### Module

In [ ]:
def load_data(
    root: str,
    batch_size: int,
    eval_batch_size: int,
    size: int = 32,
    num_workers: int = 2,
    num_ops: int = 2,
    magnitude: int = 9,
    PIN_MEMORY_FLAG: bool = True,
    PERSISTENT_WORKERS_FLAG: bool = True,
):
    '''
    # Conventional Transform
    ## For cifar-10
    mean = torch.tensor([125.3, 123.0, 113.9]) / 255
    std = torch.tensor([63.0, 62.1, 66.7]) / 255
    ## For cifar-100
    mean = torch.tensor([129.3, 124.1, 112.4]) / 255
    std = torch.tensor([68.2, 65.4, 70.4]) / 255
    ## Transform Step
    transforms.Compose([
        transforms.RandomCrop(32, padding=4, padding_mode='constant'),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    '''

    transform = {
        'train': transforms.Compose([
            transforms.Resize(size),
            transforms.RandomHorizontalFlip(),
            transforms.RandAugment(num_ops=num_ops, magnitude=magnitude),
            transforms.ToTensor(),
        ]),
        'eval': transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
        ])
    }

    dataset = {
        'train': datasets.CIFAR100(
            root=root, train=True, download=True, transform=transform['train']
        ),
        'test': datasets.CIFAR100(
            root=root, train=False, download=True, transform=transform['eval']
        )
    }

    dataloader = {
        'train': torch.utils.data.DataLoader(
            dataset['train'],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=PIN_MEMORY_FLAG,
            persistent_workers=PERSISTENT_WORKERS_FLAG
        ),
        'test': torch.utils.data.DataLoader(
            dataset['test'],
            batch_size=eval_batch_size,
            num_workers=num_workers,
            pin_memory=PIN_MEMORY_FLAG,
            persistent_workers=PERSISTENT_WORKERS_FLAG
        )
    }
    
    return dataset, dataloader

In [ ]:
## Lazy Model
class LazyBasicBlock(torch.nn.Module):
    def __init__(self, planes: int, down: bool = False) -> None:
        super().__init__()
        self.down = down
        self.conv1 = (
            torch.nn.LazyConv2d(planes, 3, stride=2, padding=1, bias=False) if down
            else torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        )
        self.bn1 = torch.nn.LazyBatchNorm2d()
        self.conv2 = torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        self.bn2 = torch.nn.LazyBatchNorm2d()
        self.relu = torch.nn.ReLU(inplace=True)
        if self.down:
            self.downsample = torch.nn.Sequential(
                torch.nn.LazyConv2d(planes, 1, stride=2, bias=False),
                torch.nn.LazyBatchNorm2d()
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.down:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class Lazy_CIFAR_ResNet(torch.nn.Module):
    def __init__(self, num_classes: int = 100, p: float = 0.2) -> None:
        super().__init__()
        self.planes = 16
        self.stem = torch.nn.Sequential(
            torch.nn.LazyConv2d(self.planes, kernel_size=3, padding='same', bias=False),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.ReLU(inplace=True)
        )
        self.layer1 = torch.nn.Sequential(
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
        )
        self.planes *= 2
        self.layer2 = torch.nn.Sequential(
            LazyBasicBlock(self.planes, down=True),
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
        )
        self.planes *= 2
        self.layer3 = torch.nn.Sequential(
            LazyBasicBlock(self.planes, down=True),
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(1),
            torch.nn.Flatten(),
            torch.nn.Dropout(p=0.2, inplace=True),
            torch.nn.LazyLinear(num_classes)
        )
    def adjust_dropout(self, p: float = 0.2):
        self.classifier[2] = torch.nn.Dropout(p=p, inplace=True)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.classifier(x)
        return x

In [ ]:
def train_step(model, dataset, dataloader, AUTOCAST_FLAG=False):
    record_loss, record_acc = 0, 0
    model.train()
    for i, data in enumerate(dataloader):
        # load data
        inputs = data[0].to(device, non_blocking=True)
        labels = data[1].to(device, non_blocking=True)
        # compute
        optimizer.zero_grad()
        with torch.autocast(device.type, enabled=AUTOCAST_FLAG):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # record
        predict_labels = torch.max(outputs, dim=1).indices
        record_loss += loss.item()
        record_acc += torch.sum(labels==predict_labels).item()
    record_loss /= len(dataloader)
    record_acc /= len(dataset)
    return record_loss, record_acc

def eval_step(model, dataset, dataloader, AUTOCAST_FLAG=False):
    record_loss, record_acc = 0, 0
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(dataloader):
            # load data
            inputs = data[0].to(device, non_blocking=True)
            labels = data[1].to(device, non_blocking=True)
            # compute
            with torch.autocast(device.type, enabled=AUTOCAST_FLAG):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            # record
            predict_labels = torch.max(outputs, dim=1).indices
            record_loss += loss.item()
            record_acc += torch.sum(labels==predict_labels).item()
    record_loss /= len(dataloader)
    record_acc /= len(dataset)
    return record_loss, record_acc

In [ ]:
def progressive_learning_scheduler(
    epoch: int,
    milestones: list,
    model: torch.nn.Module,
    dataset,
    dataloader,
    size,
    magnitude,
    p
):
    if epoch + 1 in milestones:
        size *= 2
        magnitude += 3
        p += 0.1
        dataset, dataloader = load_data(
            root=root,
            batch_size=batch_size,
            eval_batch_size=eval_batch_size,
            size=size,
            num_ops=num_ops,
            magnitude=magnitude,
            PIN_MEMORY_FLAG=PIN_MEMORY_FLAG,
            PERSISTENT_WORKERS_FLAG=PERSISTENT_WORKERS_FLAG
        )
        model.adjust_dropout(p)
    return dataset, dataloader, size, magnitude, p

def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

### Main

In [ ]:
dataset, dataloader = load_data(
    root=root,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    size=size,
    num_ops=num_ops,
    magnitude=magnitude,
    PIN_MEMORY_FLAG=PIN_MEMORY_FLAG,
    PERSISTENT_WORKERS_FLAG=PERSISTENT_WORKERS_FLAG
)

In [ ]:
model = Lazy_CIFAR_ResNet(num_classes=num_classes, p=p).to(device)

In [ ]:
scaler = torch.cuda.amp.GradScaler(
    enabled=True if device.type=='cuda' and AUTOCAST_FLAG else False
)
# compile_mode: 'default', 'reduce-overhead', 'max-autotune'
model = torch.compile(model, mode='default', fullgraph=True, disable=not COMPILE_FLAG)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay
)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

In [ ]:
for epoch in range(epochs):
    # train
    result, time_cost = timed(
        lambda: train_step(model, dataset['train'], dataloader['train'], AUTOCAST_FLAG)
    )
    train_loss, train_acc = result
    # eval
    test_loss, test_acc = eval_step(model, dataset['test'], dataloader['test'], AUTOCAST_FLAG)
    # print results
    print('----')
    print(f'epoch {epoch}')
    print(f'AUTOCAST: {AUTOCAST_FLAG}, COMPILE: {COMPILE_FLAG}')
    print(f'time_cost: {time_cost}')
    print(f'batch_size: {batch_size}')
    print(f'lr: {scheduler.get_last_lr()} / shape: {size} / magnitude: {magnitude} / dropout: {p}')
    #print(f'learning_rate: {scheduler.get_last_lr()}')
    print(f'train_loss: {train_loss}, train_acc: {train_acc}')
    print(f'test_loss: {test_loss}, test_acc: {test_acc}')
    print('----')
    
    #'''
    # scheduler
    #scheduler.step()
    '''
    # adjust image size, dropout rate
    dataset, dataloader, size, magnitude, p = progressive_learning_scheduler(
        epoch, milestones, model, dataset, dataloader, size, magnitude, p
    )
    '''